In [80]:
import numpy as np
import pandas
import re
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

#from pymystem3 import Mystem
import pymorphy2

from sklearn.model_selection import train_test_split


In [81]:
#model = Sequential()
# инициализируем
vectorizer = CountVectorizer()
#m = Mystem()
morph = pymorphy2.MorphAnalyzer()

In [82]:
# функция для лемматизации

def lemmatize(text):
    text_str = str(text)
    text_str = re.sub(r'[^\w\s]+|[\d]+', r'',text_str).strip()
    words = text_str.split() # разбиваем текст на слова
    res = ""
    for word in words:
        p = morph.parse(word)[0]
        res = res + " " + p.normal_form

    return res

In [83]:
data_xls = pandas.read_excel("resolution_100.xlsx")

In [84]:
# проводим лемматизацию
data_xls['text'] = data_xls.apply(lambda row: lemmatize(row['text']), axis=1)

In [85]:
text = data_xls.text

In [86]:
# создаем векторы
text_vector = vectorizer.fit_transform(text)

In [87]:
# чтобы получить сгенерированный словарь, из приведенной структуры CountVectorizer, стоит отметить что порядок совпадает с матрицей
vectorizer.get_feature_names()  # ['слово1', 'слово2', 'слово3', 'слово4']

['east',
 'hhru',
 'metals',
 'nan',
 'pзадача',
 'wrike',
 'аа',
 'ава',
 'аванс',
 'авр',
 'автомашина',
 'автотранспорт',
 'авцыганка',
 'агент',
 'адрес',
 'аи',
 'аккуратный',
 'акт',
 'актуализировать',
 'актуальность',
 'акцепт',
 'александр',
 'алексей',
 'алиев',
 'алкогольный',
 'алл',
 'алюминий',
 'ан',
 'анализ',
 'аналогичный',
 'аналогия',
 'анкета',
 'аннулирование',
 'аннулировать',
 'ао',
 'апрель',
 'аренда',
 'арендатор',
 'артамонов',
 'архив',
 'ас',
 'ая',
 'база',
 'базовый',
 'банк',
 'банкагарант',
 'барсук',
 'барсуков',
 'бг',
 'без',
 'белянский',
 'бесплатный',
 'битум',
 'ближний',
 'более',
 'большой',
 'боровок',
 'боровоко',
 'бородовицын',
 'бп',
 'бск',
 'бу',
 'бункеровка',
 'буссель',
 'бухгалтер',
 'бухгалтерия',
 'быстрый',
 'быть',
 'бюджет',
 'бюджетный',
 'вагон',
 'вакансия',
 'вакуумный',
 'валерий',
 'вариант',
 'ваш',
 'вводный',
 'ввоз',
 'ведение',
 'вера',
 'вернуть',
 'вернуться',
 'весь',
 'взаимодействие',
 'взаиморасчёт',
 'взять',


In [88]:
# показать матрицу
text_vector_array = text_vector.toarray()
#print(text_vector_array)

In [89]:
past_columns = [] # пустой массив, для опрееделени колонок
for i in range(len(text_vector_array[0])): 
    past_columns.append(f"v_{i}")
#past_columns

In [90]:
dataframe_vector = pandas.DataFrame(data = text_vector_array,columns = past_columns)
#dataframe_vector

In [91]:
data = pandas.concat([data_xls,dataframe_vector],axis = 1)

In [92]:
Y = data.priznak
X = data[past_columns]

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=42)

In [25]:
print(len(X_train))
print(len(X_test))

1196
300


# Sequential

In [94]:
from keras.models import Sequential
from keras.layers import Dense

In [100]:
# Создаём модель!
model = Sequential()
# Добавляем первый слой Dense, первое число 1610 - это количество нейронов, 
# input_dim - количество фич на вход
# activation -  функция активации, полулинейная функция max(x, 0) 
# именно полулинейные функции позволяют получать нелинейные результаты с минимальными затратами
model.add(Dense(1500, input_dim=1336, activation='relu'))
# добавляем второй слой с X нейронами
model.add(Dense(1000, activation='relu'))
#model.add(Dense(10, activation='relu'))
# на выходе при бинарной классификации, функцию активации чаще всего используют sigmoid , реже softmax
model.add(Dense(1, activation='sigmoid'))

# Компилирование модели. binary_crossentropy - опять же не случайно, а т.к. у нас два класса.
# Метрика accuracy используется практически для всех задач классификации
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [101]:
# Наконец дошли до обучения модели, X и Y - понятно, 
# epoch - максимальное количество эпох до остановки
# batch_size - сколько объектов будет загружаться за итерацию
model.fit(X_train, y_train, epochs=15, batch_size=128,  verbose=2)

print('\n# Оцениваем на тестовых данных')
results = model.evaluate(X_test, y_test, batch_size=10)
print('test loss, test acc:', results)

Epoch 1/15
10/10 - 0s - loss: 0.4765 - accuracy: 0.8152
Epoch 2/15
10/10 - 0s - loss: 0.1072 - accuracy: 0.9716
Epoch 3/15
10/10 - 0s - loss: 0.0414 - accuracy: 0.9891
Epoch 4/15
10/10 - 0s - loss: 0.0200 - accuracy: 0.9958
Epoch 5/15
10/10 - 0s - loss: 0.0147 - accuracy: 0.9958
Epoch 6/15
10/10 - 0s - loss: 0.0082 - accuracy: 0.9967
Epoch 7/15
10/10 - 0s - loss: 0.0076 - accuracy: 0.9975
Epoch 8/15
10/10 - 0s - loss: 0.0060 - accuracy: 0.9983
Epoch 9/15
10/10 - 0s - loss: 0.0054 - accuracy: 0.9983
Epoch 10/15
10/10 - 0s - loss: 0.0063 - accuracy: 0.9958
Epoch 11/15
10/10 - 0s - loss: 0.0060 - accuracy: 0.9983
Epoch 12/15
10/10 - 0s - loss: 0.0067 - accuracy: 0.9983
Epoch 13/15
10/10 - 0s - loss: 0.0045 - accuracy: 0.9983
Epoch 14/15
10/10 - 0s - loss: 0.0048 - accuracy: 0.9983
Epoch 15/15
10/10 - 0s - loss: 0.0048 - accuracy: 0.9975

# Оцениваем на тестовых данных
30/30 [==============================] - 0s 3ms/step - loss: 0.3603 - accuracy: 0.9233
test loss, test acc: [0.36029389500

# Input

In [113]:
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32') 

y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [114]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(1336,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(1, activation='sigmoid', name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=keras.optimizers.RMSprop(),  # Optimizer
              # Минимизируемая функция потерь
              loss=keras.losses.SparseCategoricalCrossentropy(),
              # Список метрик для мониторинга
              metrics=[keras.metrics.SparseCategoricalAccuracy()])

In [115]:
history = model.fit(X_train, y_train,
                    batch_size=64,
                    epochs=3,
                    # Мы передаем валидационные данные для
                    # мониторинга потерь и метрик на этих данных
                    # в конце каждой эпохи
                    validation_data=(X_test, y_test))

# Возвращаемый объект "history" содержит записи
# значений потерь и метрик во время обучения
#print('\nhistory dict:', history.history)

Epoch 1/3


InvalidArgumentError:  Received a label value of 1 which is outside the valid range of [0, 1).  Label values: 0 0 0 1 1 0 0 0 1 1 1 0 1 0 0 1 0 0 1 0 1 0 1 1 0 1 1 0 1 0 0 1 0 0 1 0 0 0 1 0 1 1 1 0 1 0 0 1 1 0 1 1 0 1 1 1 1 1 0 0 1 1 0 0
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-115-83c7e26cebd1>:7) ]] [Op:__inference_train_function_41038]

Function call stack:
train_function
